In [66]:
"""
Mecabによる形態素解析
hinshiリストで指定した品詞のみ抽出する
"""

import MeCab
mecab = MeCab.Tagger ()
mecab.parse('')

#形態素解析(特定の品詞のみ)
def keitaiso(text):
    node = mecab.parseToNode(text)
    word=[]
    hinshi = ["名詞","動詞","形容詞"]
    while node:
        feats = node.feature.split(',')
        if feats[0] in hinshi :
            try:
                word.append(node.surface)  #単語を取得
            except:
                print("err: " + str(node.surface))
        node = node.next  #次の単語に進める
    
    print("形態素解析(Mecab) : " + str(word), end='\n\n')
    
    return word

In [67]:
"""
KAKASIによる漢字，ひらがな，カタカナ，alfabetの相互変換
'J'漢字 'H'ひらがな　'K'カタカナ　'a'alfabet
今回は全てひらがなに変換する
"""

def kakasi(text):
    from pykakasi import kakasi
    kakasi = kakasi()
    kakasi.setMode('J', 'H')
    kakasi.setMode('K', 'H')
    kakasi.setMode('H', 'H')
    conv = kakasi.getConverter()
    kakasi_text=conv.do(text)

    print("ひらがな変換(KAKASI) : " + kakasi_text, end='\n\n')

In [71]:
"""
Cabochaによって単語を原型に変換
"""

import CaboCha

def get_word(tree, chunk):
    surface = ''
    for i in range(chunk.token_pos, chunk.token_pos + chunk.token_size):
        token = tree.token(i)
        features = token.feature.split(',')
        if features[0] == '名詞':
            surface += token.surface
        elif features[0] == '形容詞':
            surface += features[6]
            break
        elif features[0] == '動詞':
            surface += features[6]
            break
    return surface

def get_2_words(word):
    cp = CaboCha.Parser('-f1')
    tree = cp.parse(word)
    token = tree.token(0)
    chunk_dic = token.chunk
    return get_word(tree, chunk_dic)

def change_original(word):
    
    word2=get_2_words(word)
    
    if(word2!=word):
        print("原型変換(Cabocha) : " + word + " -> "+word2)
        
    return word2    

In [72]:
"""
Cabochaによって係り受け解析を行う
主語と述語の関係を示す
"""

def cabocha(line):
    cp = CaboCha.Parser('-f1')
    tree = cp.parse(line)
    chunk_dic = {}
    chunk_id = 0
    for i in range(0, tree.size()):
        token = tree.token(i)
        if token.chunk:
            chunk_dic[chunk_id] = token.chunk
            chunk_id += 1

    tuples = []
    for chunk_id, chunk in chunk_dic.items():
        if chunk.link > 0:
            from_surface =  get_word(tree, chunk)
            to_chunk = chunk_dic[chunk.link]
            to_surface = get_word(tree, to_chunk)
            tuples.append((from_surface, to_surface))
    return tuples

def kakariuke(text):
    tuples = cabocha(text)
    for t in tuples:
        print("係り受け解析(Cabocha) : " + t[0] + ' => ' + t[1])

In [74]:
if __name__ == '__main__':
    
    text = "自然言語処理は難しかった．"
    print("Original : " + text , end='\n\n')

    #通常の文字列を入力して，単語に区切る+特定の品詞のみ抽出する
    keitaiso_text = keitaiso(text)
    
    #かな漢字混じり文を入力して，すべてひらがなに変換する
    kakasi(text)
    
    #形態素解析された文章を入力して，単語を原型に変換する
    for i in range(len(keitaiso_text)):
        keitaiso_text[i] = change_original(keitaiso_text[i])
    print("原型変換(Cabocha) : " + str(keitaiso_text), end='\n\n')
    
    #通常の文字列を入力して，主語と述語の関係を得る
    kakariuke(text)

Original : 自然言語処理は難しかった．

形態素解析(Mecab) : ['自然', '言語', '処理', '難しかっ']

ひらがな変換(KAKASI) : しぜんげんごしょりはむずかしかった．

原型変換(Cabocha) : 難しかっ -> 難しい
原型変換(Cabocha) : ['自然', '言語', '処理', '難しい']

係り受け解析(Cabocha) : 自然言語処理 => 難しい
